In [1]:
import os
import sys

import requests

import numpy as np

from astropy import units as u
from astropy import table
from astropy.coordinates import SkyCoord, EarthLocation, get_sun
from astropy.time import Time
from astropy.io import fits

%matplotlib inline
from matplotlib import style, pyplot as plt

Most of the initial steps are in 2018Oct-AAT

# Initial imports/initialization of SAGA objects 

In [2]:
working_dir = os.path.abspath('..')

%cd ../../saga
sagadir = !pwd
sagadir = sagadir[0]
!python setup.py build
%cd $working_dir

/Users/erik/projects/saga/saga
running build
running build_py
copying SAGA/observing/__init__.py -> build/lib/SAGA/observing
copying SAGA/observing/aat.py -> build/lib/SAGA/observing
/Users/erik/projects/saga/erik


In [3]:
sagalibdir = os.path.join(sagadir, 'build/lib')
print('Adding', sagalibdir, 'to the python path')
sys.path.insert(0, sagalibdir)

import SAGA as saga
saga.__version__

Adding /Users/erik/projects/saga/saga/build/lib to the python path


'0.10.10'

In [4]:
saga_database = saga.Database(os.path.join(os.environ['HOME'],'Dropbox/SAGA/'))

saga_host_catalog = saga.HostCatalog(saga_database)
saga_object_catalog = saga.ObjectCatalog(saga_database)
saga_targets = saga.TargetSelection(saga_database, manual_selected_objids='manual_targets_aat2018b')#,
                                    #assign_targeting_score_kwargs=dict(ignore_specs=True))

In [5]:
hosts_to_load = 'aat_2018b'
host_id_label = 'ID_STR'
targethosts = saga_host_catalog.load(hosts_to_load)
targethosts[host_id_label] = saga_host_catalog.resolve_id(hosts_to_load, 'string')


# used below to save generated catalogs
acats = {}
tcats = {}

In [6]:
# helper scripts for AAT

from SAGA.observing import (get_gaia_guidestars, get_sdss_guidestars, 
                            subsample_catalog, write_fld_file, 
                            make_decals_viewer_cutouts, 
                            make_des_cutouts_file, show_des_cutouts)

# Configuration File-Generation Templates 

## nsaxxxx

In [ ]:
idnum = xxxx
fieldnum = NN

saga_object_catalog.build_and_write_to_database('NSA{}'.format(idnum), overwrite=fieldnum>1)

In [ ]:
host = targethosts[targethosts['NSAID'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_sdss_guidestars(hostname)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

In [ ]:
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))], 
                           survey='sdss',ncols=6,namecol='TargetName')

In [ ]:
guides_to_rem = [int(i) for i in 'nnnn nnn nnnn'.split()]
writecat.remove_rows((writecat['Notes']=='Guide')&(np.in1d(writecat['TargetName'], guides_to_rem)))
np.sum((writecat['Notes']=='Guide'))

In [ ]:
make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='sdss', ncols=6, 
                           zoom=16, namecol='TargetName')

In [ ]:
skys_to_rem = [int(i) for i in 'n n nn nn'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

In [ ]:
tt = writecat['TargetName', 'TargetType']
tt['TargetName'].name = 'OBJID'
tt['idx'] = np.arange(len(tt))
joined_cat = table.join(tt, tcat, join_type='left')
joined_cat = joined_cat[np.argsort(joined_cat['idx'])]

# edit 1:... to limit the number to something reasonable if necessary
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100, 
                                  favor_highpri=joined_cat['TARGETING_SCORE'])

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

In [ ]:
!scp $fldfn visitor8@aatlxh:~/octDD/

## pgcxxxx

In [ ]:
idnum = xxxx
fieldnum = NN

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

In [ ]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname, saga_host_catalog, saga_object_catalog)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

In [ ]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

In [ ]:
#The commented part below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

In [ ]:
skys_to_rem = [int(i) for i in <'n n nn'>.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

In [ ]:
tt = writecat['TargetName', 'TargetType']
tt['TargetName'].name = 'OBJID'
tt['idx'] = np.arange(len(tt))
joined_cat = table.join(tt, tcat, join_type='left')
joined_cat = joined_cat[np.argsort(joined_cat['idx'])]

# edit 1:... to limit the number to something reasonable if necessary
subsampledcat = subsample_catalog(writecat, {1:np.inf}, maxflux=5, maxguides=100, 
                                  favor_highpri=joined_cat['TARGETING_SCORE'])

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

In [ ]:
!scp $fldfn visitor8@aatlxh:~/octDD/

# Night 3

In [7]:
import aat

obs_start_date = Time('2018-8-5') 

## pgc67663

In [8]:
idnum = 67663
fieldnum = 2  # 1 was done in a previous run

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>2)

[10/05 15:25:41] [WARNING] Not found: SDSS catalog for pgc67663.
[10/05 15:25:41] [WARNING] Not found: DECALS catalog for pgc67663.
[10/05 15:25:41] Use DES to build base catalog v2 for pgc67663 (1/1)


[10/05 15:26:04] Write base catalog to ./base_catalogs/base_v2_pgc67663.fits.gz


In [23]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname, saga_host_catalog, saga_object_catalog)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc67663
21h56m09.6s -49d31m19.2s
# of flux stars = 207
# of sky fibers = 100
# of Priority=1 targets = 785
# of Priority=2 targets = 130
# of Priority=3 targets = 29
# of Priority=4 targets = 50
# of Priority=5 targets = 158
# of Priority=6 targets = 102
# of Priority=7 targets = 109
# of Priority=8 targets = 11
# of Priority=9 targets = 307
Object catalog to Gaia offset: (<Angle -0.12399641 arcsec>, <Angle 0.14305041 arcsec>) from 1117 objects (22.2%)
Found 187 Gaia guide stars
63 Have a brightish neighbor.  Removing them.


1805

In [24]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

6559435790036322432,6559427440619902976,6560708990141953024,6558354931450820608,6558997802155435264,6559378752870600960
6559197191716574336,6559078380037754752,6560672908119653632,6559862228749567104,6559366555163475968,6559138887537143808
6560901920072537344,6559432354062474880,6559195443666525952,6560690362866747776,6559458467463704192,6560729739126905600
6559178813553069440,6559165275816192640,6560914289578381696,6559264712899003392,6560726857205878656,6559266430886011648


In [25]:
#The commented part below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [26]:
skys_to_rem = [int(i) for i in '2 23 27 33 47 51 62 73 75 83'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

90

In [27]:
subsampledcat = subsample_catalog(writecat, {1:200}, maxflux=5, maxguides=100)

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 100 NSky 90
Targets in each priority: {1: 200, 2: 130, 3: 29, 4: 50, 5: 158, 6: 102, 7: 109, 8: 11}
Total: 984
Writing aat_targets_oct2018/pgc67663_2.fld


789

In [40]:
!scp $fldfn visitor8@aatlxh:~/oct05/

pgc67663_2.fld                                100%   53KB  53.3KB/s   00:00    


## pgc1952

In [29]:
idnum = 1952
fieldnum = 1

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

[10/05 15:36:29] Base catalog v2 for pgc1952 already exists (./base_catalogs/base_v2_pgc1952.fits.gz). (1/1)


In [30]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname, saga_host_catalog, saga_object_catalog)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc1952
00h32m02.112s -64d15m12.24s
# of flux stars = 141
# of sky fibers = 100
# of Priority=1 targets = 503
# of Priority=2 targets = 128
# of Priority=3 targets = 10
# of Priority=4 targets = 50
# of Priority=5 targets = 118
# of Priority=6 targets = 63
# of Priority=7 targets = 48
# of Priority=8 targets = 6
# of Priority=9 targets = 241
Object catalog to Gaia offset: (<Angle -0.36644631 arcsec>, <Angle 0.07111843 arcsec>) from 921 objects (72.6%)
Found 171 Gaia guide stars
46 Have a brightish neighbor.  Removing them.


1292

In [31]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

4708465482876995712,4708510803371773184,4900507214576396544,4708372161827678464,4900546247239120768,4900619467841556480
4708170401444068608,4900613454887348864,4708189608537796736,4708484479516380160,4900660218491329408,4708480257564404480
4900772506116151552,4708226648335664512,4708367660701955328,4900612733332849280,4708216202975276416,4708478677016445184
4708313544114121344,4708273373284028928,4900566519484787328,4708402776354414720,4708190914207846400,4900813084967138304


In [32]:
#The commented part below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [33]:
skys_to_rem = [int(i) for i in '1 5 9 11 15 16 18 20 22 24 27 29 35 41 43 45 50 55 57 58 59 60 62 65 67 69 71 72 75 76 77 79 84 90 91 93 96 97 98'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

61

In [50]:
tt = writecat['TargetName', 'TargetType']
tt['TargetName'].name = 'OBJID'
tt['idx'] = np.arange(len(tt))
joined_cat = table.join(tt, tcat, join_type='left')
joined_cat = joined_cat[np.argsort(joined_cat['idx'])]

subsampledcat = subsample_catalog(writecat, {1:200}, maxflux=5, maxguides=100, 
                                  favor_highpri=joined_cat['TARGETING_SCORE'])

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 100 NSky 61
Targets in each priority: {1: 200, 2: 128, 3: 10, 4: 50, 5: 118, 6: 63, 7: 48, 8: 6}
Total: 789
Writing aat_targets_oct2018/pgc1952_1.fld


623

In [51]:
!scp $fldfn visitor8@aatlxh:~/oct05/

pgc1952_1.fld                                 100%   42KB  42.4KB/s   00:00    


## pgc9747

In [52]:
idnum = 9747
fieldnum = 1

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

[10/05 20:56:00] Base catalog v2 for pgc9747 already exists (./base_catalogs/base_v2_pgc9747.fits.gz). (1/1)


In [53]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname, saga_host_catalog, saga_object_catalog)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc9747
02h33m34.296s -39d02m42s
# of flux stars = 61
# of sky fibers = 100
# of Priority=1 targets = 783
# of Priority=2 targets = 44
# of Priority=3 targets = 6
# of Priority=4 targets = 50
# of Priority=5 targets = 182
# of Priority=6 targets = 62
# of Priority=7 targets = 34
# of Priority=8 targets = 9
# of Priority=9 targets = 161
Object catalog to Gaia offset: (<Angle -0.20728011 arcsec>, <Angle 0.01741113 arcsec>) from 577 objects (63.3%)
Found 118 Gaia guide stars
16 Have a brightish neighbor.  Removing them.


1433

In [54]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

4952928664814001152,4952840257207241728,4953143344459198976,4953110840146633344,4952883790995702528,4952981716249928704
4952901417541455360,4952893068125039744,4953415748464884608,4952994601151818496,4952922995457173120,4953181655567438464
4952945054409167488,4953266111804228992,4953165712648862080,4952228825663178112,4953278446950296704,4952627467347811200
4953144821927944448,4953256418061845760,4952989481550797312,4953082046685901184,4952961718882264064,4952877984199919360


In [55]:
#The commented part below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [56]:
skys_to_rem = [int(i) for i in '6 12 21 32 37 50 58 59 82 89 99'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

89

In [57]:
tt = writecat['TargetName', 'TargetType']
tt['TargetName'].name = 'OBJID'
tt['idx'] = np.arange(len(tt))
joined_cat = table.join(tt, tcat, join_type='left')
joined_cat = joined_cat[np.argsort(joined_cat['idx'])]

subsampledcat = subsample_catalog(writecat, {1:200}, maxflux=5, maxguides=100, 
                                  favor_highpri=joined_cat['TARGETING_SCORE'])

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 100 NSky 89
Targets in each priority: {1: 200, 2: 44, 3: 6, 4: 50, 5: 182, 6: 62, 7: 34, 8: 9}
Total: 781
Writing aat_targets_oct2018/pgc9747_1.fld


587

In [58]:
!scp $fldfn visitor8@aatlxh:~/oct05/

pgc9747_1.fld                                 100%   42KB  41.6KB/s   00:00    


## pgc10965

In [59]:
idnum = 10965
fieldnum = 1

saga_object_catalog.build_and_write_to_database('pgc{}'.format(idnum), overwrite=fieldnum>1)

[10/05 22:04:52] Base catalog v2 for pgc10965 already exists (./base_catalogs/base_v2_pgc10965.fits.gz). (1/1)


In [60]:
host = targethosts[targethosts['PGC'] == idnum]
assert len(host) == 1
host = host[0]

hostname = host[host_id_label]
print(hostname)
print(SkyCoord(host['RA'], host['Dec'], unit=u.deg).to_string('hmsdms'))

tcats[hostname] = tcat = saga_targets.build_target_catalogs(hostname, return_as='list', reload_base=True).pop()
acats[hostname] = acat = saga.targets.prepare_aat_catalog(tcat, sky_fiber_needed=100)

guidestars = get_gaia_guidestars(hostname, saga_host_catalog, saga_object_catalog)
writecat = table.vstack([acat, guidestars])
writecat['Notes'] = [n.title() for n in writecat['Notes']]
len(writecat)

pgc10965
02h54m33.48s -18d38m06s


/Users/erik/miniconda3/envs/saga3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


# of flux stars = 53
# of sky fibers = 100
# of Priority=1 targets = 667
# of Priority=2 targets = 76
# of Priority=3 targets = 22
# of Priority=4 targets = 50
# of Priority=5 targets = 126
# of Priority=6 targets = 92
# of Priority=7 targets = 39
# of Priority=8 targets = 4
# of Priority=9 targets = 153


/Users/erik/miniconda3/envs/saga3/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)
/Users/erik/miniconda3/envs/saga3/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


Object catalog to Gaia offset: (<Angle -0.1295607 arcsec>, <Angle 0.0761455 arcsec>) from 592 objects (70.5%)
Found 118 Gaia guide stars
26 Have a brightish neighbor.  Removing them.


1321

In [61]:
# only need to remove if something really fishy is going on
make_decals_viewer_cutouts(guidestars[np.random.permutation(len(guidestars))[:24]], 
                           survey='unwise-w1w2',ncols=6,namecol='TargetName', zoom=17)

5152194527209060864,5153246862915935488,5152511427075901312,5128137663765554944,5152510980399700992,5152524281913703168
5129209275285512448,5129217933939573760,5128412472953061248,5128046571803441280,5128244659990674432,5128447760404375936
5128176799507528960,5152452951096870784,5128153125647799936,5128417695633311616,5128465421309896192,5128426487430512896
5152215349210774912,5128237302711063936,5128174875362191872,5128402298175011584,5129243944261510144,5128144948030070016


In [62]:
#The commented part below is only necessary if the decals viewer is broken but the targets are in DES
# make_des_cutouts_file(acat[acat['Notes']=='sky'])
# show_des_cutouts(acat[acat['Notes']=='sky'], <'JOBNAME'>, ncols=8, namecol='TargetName')

make_decals_viewer_cutouts(acat[acat['Notes']=='sky'], survey='des-dr1', namecol='TargetName',
                           ncols=8, zoom=15, size=120)

0,1,2,3,4,5,6,7
8,9,10,11,12,13,14,15
16,17,18,19,20,21,22,23
24,25,26,27,28,29,30,31
32,33,34,35,36,37,38,39
40,41,42,43,44,45,46,47
48,49,50,51,52,53,54,55
56,57,58,59,60,61,62,63
64,65,66,67,68,69,70,71
72,73,74,75,76,77,78,79
80,81,82,83,84,85,86,87


In [63]:
skys_to_rem = [int(i) for i in '10 19 20 23 27 28 35 36 37 41 45 50 53 55 58 62 67 71 75 77 81 89 91 96'.split()]
writecat.remove_rows((writecat['Notes']=='Sky')&(np.in1d(writecat['TargetName'], skys_to_rem)))
np.sum((writecat['Notes']=='Sky'))

76

In [65]:
tt = writecat['TargetName', 'TargetType']
tt['TargetName'].name = 'OBJID'
tt['idx'] = np.arange(len(tt))
joined_cat = table.join(tt, tcat, join_type='left')
joined_cat = joined_cat[np.argsort(joined_cat['idx'])]

# edit 1:... to limit the number to something reasonable if necessary
subsampledcat = subsample_catalog(writecat, {1:200}, maxflux=5, maxguides=100, 
                                  favor_highpri=joined_cat['TARGETING_SCORE'])

fldfn = 'aat_targets_oct2018/{}_{}.fld'.format(hostname, fieldnum)
print('Writing', fldfn)
write_fld_file(subsampledcat, host, obs_start_date, fldfn, suffix='_{}'.format(fieldnum))
np.sum(subsampledcat['Notes'] == 'Targets')

Nflux: 5 Nguide: 92 NSky 76
Targets in each priority: {1: 200, 2: 76, 3: 22, 4: 50, 5: 126, 6: 92, 7: 39, 8: 4}
Total: 782
Writing aat_targets_oct2018/pgc10965_1.fld


609

In [66]:
!scp $fldfn visitor8@aatlxh:~/oct05/

pgc10965_1.fld                                100%   42KB  41.8KB/s   00:00    


# Scratch/working space 

## Check fibers from list file (mostly for guidestar when on sky) 

In [ ]:
lt, sc, _ = aat.load_lis_file('aat_targets_oct2018/nsa13927_1_p0.lis')
lt['ra'] = sc.ra.deg
lt['dec'] = sc.dec.deg

In [ ]:
make_decals_viewer_cutouts(lt[lt['codes']=='F'], 'sdss', namecol='fibnums', ncols=4)

In [ ]:
make_decals_viewer_cutouts(lt[lt['codes']=='F'], 'unwise-w1w2', namecol='mags', ncols=4, zoom=17)